<a href="https://colab.research.google.com/github/AfrarJahin/WSD/blob/main/Wsd_fine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/data9k.csv"

In [ ]:
!pip install BnVec
# !git clone https://github.com/banglakit/bengali-stemmer.git
# !python -m pip install --upgrade pip
# %cd bengali-stemmer
# !pip install flake8

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import seaborn as sns
import pickle
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from  keras . utils  import  to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D,Conv2D, MaxPooling1D, Embedding, Flatten
from keras.models import Model
from keras.initializers import Constant
from keras.layers import Embedding


from pandas import DataFrame
# from BnVec import CountVectorizer

#Load Files

In [ ]:
df = pd.read_csv(path)
s=len(set(df['ambiguous_word']))


In [ ]:
df.head()

In [ ]:
text = list(df['Text'])

In [ ]:
ambi_word = list(df['ambiguous_word'])

In [ ]:
type(ambi_word)

list

In [ ]:
text[0]

'মাথা হলো মানবদেহে বিদ্যমান সবচেয়ে ভারি অংশ। মাথা'

In [ ]:
# from bengali_stemmer.rafikamal2014 import RafiStemmer
# stemmer = RafiStemmer()

In [ ]:
# from bengali_stemmer.rafikamal2014 import RafiStemmer
# stemmer = RafiStemmer()
# stemmer.stem_word('কাটতেছে')

In [ ]:
# df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/fine_tuned_data.csv')
stopwords = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset/Stopwords.csv')

In [ ]:
# length_array2 = []
# for entry in df['Text']:
#   length_array2.append(len(entry))


In [ ]:
# length_array2 = np.array(length_array2)

In [ ]:
# df = df.drop(['Unnamed: 3'] , axis=1)

In [ ]:
set_stop = set(stopwords['words'])

#Data Cleaning

In [ ]:
def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence)
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
    # cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned1)
    return cleaned1

In [ ]:
def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    all_negative_words = []
    s=''

    for sentence in data:
        filtered_sentence = []
        for w in str(sentence).split():
            for cleaned_word in clean_punct(w).split():
                if len(cleaned_word)>2:
                    # print(cleaned_word)
                    if((cleaned_word) not in set_stop):
                        # s = stemmer.stem_word(cleaned_word)
                        s = cleaned_word
                        #print(s)
                        if len(s)>2:
                            final_words.append(s)
                            filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue

        str1 = " ".join(filtered_sentence)
        final_string.append(str1)
    return final_string


In [ ]:
import re
re.compile('<title>(.*)</title>')
text = pre_process(df['Text'])



In [ ]:
# len(set(df['ambiguous_word']))

33

In [ ]:
# type(text)

list

In [ ]:
# type(ambi_word)

list

In [ ]:
for i in range(len(text)):
	text[i] = str(text[i])

for i in range(len(ambi_word)):
	ambi_word[i] = str(ambi_word[i])

In [ ]:
for i in range(len(text)):
   ambi_word 

In [ ]:
# type(df['ambiguous_word'][0])

str

In [ ]:

for i in range(len(text)):
  text[i] = text[i] + df['ambiguous_word'][i]
  


In [ ]:
import tensorflow as tf

In [ ]:
from keras.preprocessing.text import Tokenizer
# define 5 documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!']
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(text)

In [ ]:
def load_data(num_words, sequence_length, test_size=0.25, oov_token=None):
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
    num_classes=16
    tokenizer.fit_on_texts(texts)
    X = tokenizer.texts_to_sequences(texts)
    X = np.array(X)
    # pad sequences with 0's
    X = pad_sequences(X, maxlen=sequence_length)
    # split data to training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)
    data = {}
    data["X_train"] = X_train
    data["X_test"]= X_test
    # data["y_train"] = y_train
    # data["y_test"] = y_test
    data["tokenizer"] = tokenizer
    return data

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [ ]:

def get_embedding_vectors(word_index, embedding_size=100):
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_size))
    # with open(f"kaggle/input/bangla-golve/bn_glove.{embedding_size}d.txt", encoding="utf8") as f:
    with open(f"/content/drive/MyDrive/Colab Notebooks/Dataset/bn_glove.{embedding_size}d.txt", encoding="utf8") as f:
        for line in tqdm(f, "Reading GloVe"):
            values = line.split()
            # get the word as the first word in the line
            word = values[0]
            if word in word_index:
                idx = word_index[word]
                # get the vectors as the remaining values in the line
                embedding_matrix[idx] = np.array(values[1:], dtype="float32")
    return embedding_matrix

In [ ]:
vec = get_embedding_vectors(t.word_index)

In [ ]:
embedding_vecor_length = 300
model = Sequential()
model.add(Embedding(14028, 100,  weights=[vec], input_length=100))
model.add(LSTM(100))
model.add(Dense(16, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
history = model.fit(text, label, validation_split=0.25  ,epochs=10, verbose=2)

In [ ]:
# define model

model = Sequential()
e = Embedding(vocab_size, 100, weights=[vec], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(33, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

#Tokenization

In [ ]:
pip install bnlp_toolkit

In [ ]:
from bnlp import BasicTokenizer     

 #for text tokenization    
basic_t = BasicTokenizer()
raw_text = []
for i in range(len(texts)):
  tokens = basic_t.tokenize(texts[i])
  raw_text.append(tokens)
raw_text



In [ ]:
raw_text

In [ ]:
text = pre_process(raw_text)

In [ ]:
#for ambiguous_word tokenization
raw_ambi = []
for i in range(len(ambi)):
  tokens = basic_t.tokenize(ambi[i])
  raw_ambi.append(tokens)
raw_ambi

In [ ]:
#for meaning tokenization
raw_mean = []
for i in range(len(label)):
  tokens = basic_t.tokenize(label[i])
  raw_mean.append(tokens)
raw_mean

In [ ]:
#POS Tagging text only!
from bnlp import POS

bn_pos = POS()
model_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/bn_pos.pkl"

res = []
for i in range(len(text)):
  p = bn_pos.tag(model_path,text[i])
  res.append(p)




In [ ]:
# from bnlp import BengaliGlove

# bng = BengaliGlove()
# glove_path = "/content/drive/MyDrive/Colab Notebooks/Dataset/bn_glove.100d.txt"
# word = "গ্রাম"
# res = bng.closest_word(glove_path, word)
# print(res)
# vec = bng.word2vec(glove_path, word)
# print(vec)

In [ ]:
df['ambi_word']= raw_ambi
df['sense']= raw_mean
df['text']= raw_text

In [ ]:
df

In [ ]:
X= df.drop(['ambiguous_word','meaning','Text','sense'], axis=1)

y = df['sense']

In [ ]:
X.head(4)

In [ ]:
y.head()

In [ ]:
ambi_word = list(X['ambi_word'])
text = list(X['text'])

In [ ]:
text

In [ ]:
ambi_word

#Count Vectorization

In [ ]:
from pandas import DataFrame
from BnVec import CountVectorizer
cv = CountVectorizer()
x_ambi = cv.fit_transform(ambi_word) # x is the word features
x_ambi.shape

In [ ]:
print(cv.get_feature_names())

In [ ]:
x_text = cv.fit_transform(text)
x_text.shape

In [ ]:
x = np.concatenate((x_text, x_ambi), 1)

In [ ]:
y = df['meaning']

In [ ]:
y

In [ ]:
macronum=sorted(set(y.astype(str)))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

label=y.apply(fun)

In [ ]:
y=label
y.shape

In [ ]:
# with open('/content/drive/MyDrive/Colab Notebooks/Dataset/.txt', 'wb') as f:
#     pickle.dump(y, f)

In [ ]:
# with open('/content/drive/MyDrive/Colab Notebooks/Dataset/data.txt', 'rb') as f:
#     x = pickle.load(f)

# **SVM**

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

In [ ]:
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
clf.score(X_test, y_test)

In [ ]:
svm_pred = clf.predict(X_test)
svm_acc = accuracy_score(y_test, svm_pred, normalize=True) * float(100)

print('SVM Accuracy %d%%' % (svm_acc))

In [ ]:
# from sklearn.model_selection import cross_val_score
# clf = svm.SVC(kernel='linear', C=1)
# scores = cross_val_score(clf, x, y, cv=5)
# scores

In [ ]:
# from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# print(confusion_matrix(y_cv,svm_pred))
# print(classification_report(y_cv,svm_pred))

##KNN

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization purposes
import seaborn as sns # for data visualization
%matplotlib inline

In [ ]:
# import KNeighbors ClaSSifier from sklearn
from sklearn.neighbors import KNeighborsClassifier

# instantiate the model
knn = KNeighborsClassifier(n_neighbors=3)


# instantiate the model
knn = KNeighborsClassifier(n_neighbors=3)


# fit the model to the training set
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)

# y_pred

In [ ]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
y_pred_train = knn.predict(X_train)

In [ ]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

In [ ]:
from sklearn import metrics

In [ ]:
# experimenting with different n values
k_range = list(range(1,26))
scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    scores.append(metrics.accuracy_score(y_test, y_pred))
    
plt.plot(k_range, scores)
plt.xlabel('Value of k for KNN')
plt.ylabel('Accuracy Score')
plt.title('Accuracy Scores for Values of k of k-Nearest-Neighbors')
plt.show()

In [ ]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(knn.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(knn.score(X_test, y_test)))

In [ ]:
# instantiate the model with k=5
knn_5 = KNeighborsClassifier(n_neighbors=5)


# fit the model to the training set
knn_5.fit(X_train, y_train)


# predict on the test-set
y_pred_5 = knn_5.predict(X_test)


print('Model accuracy score with k=5 : {0:0.4f}'. format(accuracy_score(y_test, y_pred_5)))

#Load Dataset

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Dataset/target.txt', 'rb') as f:
    y = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Dataset/bn_pos.pkl', 'rb') as f:
    pos = pickle.load(f)

In [ ]:
print(pos)

In [ ]:
# split X and y into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)


In [ ]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

##Naive Bayes


In [ ]:
# train a Gaussian Naive Bayes classifier on the training set
from sklearn.naive_bayes import GaussianNB


# instantiate the model
gnb = GaussianNB()


#fit the model
gnb.fit(X_train, y_train)

In [ ]:
y_pred = gnb.predict(X_test)

#y_pred

In [ ]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
y_pred_train = gnb.predict(X_train)

#y_pred_train

In [ ]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

In [ ]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(gnb.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(gnb.score(X_test, y_test)))

In [ ]:
# Applying 10-Fold Cross Validation

from sklearn.model_selection import cross_val_score

scores = cross_val_score(gnb, X_train, y_train, cv = 10, scoring='accuracy')

print('Cross-validation scores:{}'.format(scores))

In [ ]:
# compute Average cross-validation score

print('Average cross-validation score: {:.4f}'.format(scores.mean()))

##KNN

In [ ]:
# import KNeighbors ClaSSifier from sklearn
from sklearn.neighbors import KNeighborsClassifier


# instantiate the model
knn = KNeighborsClassifier(n_neighbors=3)


# fit the model to the training set
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)

#y_pred

In [ ]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
y_pred_train = knn.predict(X_train)

In [ ]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

In [ ]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(knn.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(knn.score(X_test, y_test)))

In [ ]:
# instantiate the model with k=5
knn_5 = KNeighborsClassifier(n_neighbors=5)


# fit the model to the training set
knn_5.fit(X_train, y_train)


# predict on the test-set
y_pred_5 = knn_5.predict(X_test)


print('Model accuracy score with k=5 : {0:0.4f}'. format(accuracy_score(y_test, y_pred_5)))

In [ ]:

# instantiate the model with k=7
knn_7 = KNeighborsClassifier(n_neighbors=7)


# fit the model to the training set
knn_7.fit(X_train, y_train)


# predict on the test-set
y_pred_7 = knn_7.predict(X_test)


print('Model accuracy score with k=7 : {0:0.4f}'. format(accuracy_score(y_test, y_pred_7)))

In [ ]:
# Applying 10-Fold Cross Validation

from sklearn.model_selection import cross_val_score

scores = cross_val_score(knn, X_train, y_train, cv = 10, scoring='accuracy')

print('Cross-validation scores:{}'.format(scores))

In [ ]:
# compute Average cross-validation score

print('Average cross-validation score: {:.4f}'.format(scores.mean()))

##SVM

In [ ]:
# import SVC classifier
from sklearn.svm import SVC


# import metrics to compute accuracy
from sklearn.metrics import accuracy_score


# instantiate classifier with default hyperparameters
svc=SVC() 


# fit classifier to training set
svc.fit(X_train,y_train)


# make predictions on test set
y_pred=svc.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
print('Training set score: {:.4f}'.format(svc.score(X_train, y_train)))

In [ ]:
# Applying 10-Fold Cross Validation

from sklearn.model_selection import cross_val_score

s_scores = cross_val_score(linear_svc, X_train, y_train, cv = 10, scoring='accuracy')

print('Cross-validation scores:{}'.format(s_scores))

In [ ]:
# compute Average cross-validation score

print('Average cross-validation score: {:.4f}'.format(s_scores.mean()))

In [ ]:
# instantiate classifier with linear kernel and C=1.0
linear_svc=SVC(kernel='linear', C=1.0) 


# fit classifier to training set
linear_svc.fit(X_train,y_train)


# make predictions on test set
y_pred_test=linear_svc.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with linear kernel and C=1.0 : {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

In [ ]:
# instantiate classifier with rbf kernel and C=100
svc=SVC(C=100.0) 


# fit classifier to training set
svc.fit(X_train,y_train)


# make predictions on test set
y_pred=svc.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with rbf kernel and C=100.0 : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
# instantiate classifier with rbf kernel and C=1000
svc=SVC(C=1000.0) 


# fit classifier to training set
svc.fit(X_train,y_train)


# make predictions on test set
y_pred=svc.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with rbf kernel and C=1000.0 : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
# instantiate classifier with linear kernel and C=100.0
linear_svc100=SVC(kernel='linear', C=100.0) 


# fit classifier to training set
linear_svc100.fit(X_train, y_train)


# make predictions on test set
y_pred=linear_svc100.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with linear kernel and C=100.0 : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
# instantiate classifier with linear kernel and C=1000.0
linear_svc1000=SVC(kernel='linear', C=1000.0) 


# fit classifier to training set
linear_svc1000.fit(X_train, y_train)


# make predictions on test set
y_pred=linear_svc1000.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with linear kernel and C=1000.0 : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
y_pred_train = linear_svc.predict(X_train)

y_pred_train

In [ ]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

In [ ]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(linear_svc.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(linear_svc.score(X_test, y_test)))

In [ ]:
# instantiate classifier with polynomial kernel and C=1.0
poly_svc=SVC(kernel='poly', C=1.0) 


# fit classifier to training set
poly_svc.fit(X_train,y_train)


# make predictions on test set
y_pred=poly_svc.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with polynomial kernel and C=1.0 : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
# instantiate classifier with polynomial kernel and C=100.0
poly_svc100=SVC(kernel='poly', C=100.0) 


# fit classifier to training set
poly_svc100.fit(X_train, y_train)


# make predictions on test set
y_pred=poly_svc100.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with polynomial kernel and C=1.0 : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:

# instantiate classifier with sigmoid kernel and C=1.0
sigmoid_svc=SVC(kernel='sigmoid', C=1.0) 


# fit classifier to training set
sigmoid_svc.fit(X_train,y_train)


# make predictions on test set
y_pred=sigmoid_svc.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with sigmoid kernel and C=1.0 : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
# instantiate classifier with sigmoid kernel and C=100.0
sigmoid_svc100=SVC(kernel='sigmoid', C=100.0) 


# fit classifier to training set
sigmoid_svc100.fit(X_train,y_train)


# make predictions on test set
y_pred=sigmoid_svc100.predict(X_test)


# compute and print accuracy score
print('Model accuracy score with sigmoid kernel and C=100.0 : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
# import GridSearchCV
from sklearn.model_selection import GridSearchCV


# import SVC classifier
from sklearn.svm import SVC


# instantiate classifier with default hyperparameters with kernel=rbf, C=1.0 and gamma=auto
svc=SVC() 



# declare parameters for hyperparameter tuning
parameters = [ {'C':[1, 10, 100, 1000], 'kernel':['linear']},
               {'C':[1, 10, 100, 1000], 'kernel':['rbf'], 'gamma':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
               {'C':[1, 10, 100, 1000], 'kernel':['poly'], 'degree': [2,3,4] ,'gamma':[0.01,0.02,0.03,0.04,0.05]} 
              ]




grid_search = GridSearchCV(estimator = svc,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=0)


grid_search.fit(X_train, y_train)

In [ ]:
# examine the best model


# best score achieved during the GridSearchCV
print('GridSearch CV best score : {:.4f}\n\n'.format(grid_search.best_score_))


# print parameters that give the best results
print('Parameters that give the best results :','\n\n', (grid_search.best_params_))


# print estimator that was chosen by the GridSearch
print('\n\nEstimator that was chosen by the search :','\n\n', (grid_search.best_estimator_))

# **Naive Bayes**

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state= 0)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
grid={'C':10.0 **np.arange(-2,3),'penalty':['l1','l2']}
cv=KFold(n_splits=5,random_state=None,shuffle=False)

In [ ]:
clf=GridSearchCV(gnb,grid,cv=cv,n_jobs=-1,scoring='f1_macro')
clf.fit(X_train,y_train)

In [ ]:
estimator.get_params().keys()

In [ ]:
#Predict the results
y_pred = gnb.predict(X_test)

y_pred

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
#Check accuracy score
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
precision_score(y_train, y_pred)

In [ ]:
recall_score(y_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_pred)

In [ ]:
###Compare the train-set and test-set accuracy
y_pred_train = gnb.predict(X_train)

y_pred_train

In [ ]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

In [ ]:
##Check for overfitting and underfitting
print('Training set score: {:.4f}'.format(gnb.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(gnb.score(X_test, y_test)))

In [ ]:
###Null Accuracy
# check class distribution in test set

y_test.value_counts()

In [ ]:
# check null accuracy score

null_accuracy = (***/(***+***))

print('Null accuracy score: {0:0.4f}'. format(null_accuracy))

In [ ]:
##Confusion Metrics

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

print('Confusion matrix\n\n', cm)

print('\nTrue Positives(TP) = ', cm[0,0])

print('\nTrue Negatives(TN) = ', cm[1,1])

print('\nFalse Positives(FP) = ', cm[0,1])

print('\nFalse Negatives(FN) = ', cm[1,0])

In [ ]:
#Classification metrices


from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
# # visualize confusion matrix with seaborn heatmap

# cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
#                                  index=['Predict Positive:1', 'Predict Negative:0'])

# sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

In [ ]:
# Applying 10-Fold Cross Validation

from sklearn.model_selection import cross_val_score

scores = cross_val_score(gnb, X_train, y_train, cv = 10, scoring='accuracy')

print('Cross-validation scores:{}'.format(scores))

In [ ]:
scores = cross_val_score(gnb, x, y, cv = 10, scoring='accuracy')
scores

In [ ]:
print(scores.mean())

In [ ]:
# compute Average cross-validation score

print('Average cross-validation score: {:.4f}'.format(scores.mean()))

In [ ]:
# Applying 10-Fold Cross Validation

from sklearn.model_selection import cross_val_score

t_scores = cross_val_score(gnb, X_test, y_test, cv = 10, scoring='accuracy')

print('Cross-validation scores:{}'.format(scores))

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x, y);

In [ ]:
plt.scatter(x[:, 0], x[:, 1], c=y, s=50, cmap='RdBu');

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(test.target, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=train.target_names, yticklabels=train.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');

## Further evaluation with SVC base estimator 

In [ ]:
# load required classifer
from sklearn.ensemble import AdaBoostClassifier


# import Support Vector Classifier
from sklearn.svm import SVC


# import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import accuracy_score
svc=SVC(probability=True, kernel='linear')


# create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1, random_state=0)


# train adaboost classifer
model2 = abc.fit(X_train, y_train)


# predict the response for test dataset
y_pred = model2.predict(X_test)


# calculate and print model accuracy
print("Model Accuracy with SVC Base Estimator:",accuracy_score(y_test, y_pred))

##Evaluate Model

In [ ]:
#import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import accuracy_score


# calculate and print model accuracy
print("AdaBoost Classifier Model Accuracy:", accuracy_score(y_test, y_pred))

##**Adaboost Model**

In [ ]:
# Import the AdaBoost classifier
from sklearn.ensemble import AdaBoostClassifier


# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1, random_state=0)

# Train Adaboost Classifer
model1 = abc.fit(X_train, y_train)


#Predict the response for test dataset
y_pred = model1.predict(X_test)

#LSTM

In [ ]:
def get_embedding_vectors(word_index, embedding_size=100):
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_size))
    with open(f"/content/drive/MyDrive/Colab Notebooks/Dataset/bn_glove.{embedding_size}d.txt", encoding="utf8") as f:
        for line in tqdm(f, "Reading GloVe"):
            values = line.split()
            # get the word as the first word in the line
            word = values[0]
            if word in word_index:
                idx = word_index[word]
                # get the vectors as the remaining values in the line
                embedding_matrix[idx] = np.array(values[1:], dtype="float32")
    return embedding_matrix

In [ ]:
embedding_matrix = get_embedding_vectors(corpus)

In [ ]:
embedding_vecor_length = 100
MAX_NUM_WORDS = 8411
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, embedding_vecor_length,  weights=[embedding_matrix], input_length=max_sequence_length))
model.add(LSTM(100))

model.add(Dense(528, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(x=train_x,y=train_y,batch_size=100,epochs=10,verbose=1,shuffle=True,validation_split=0.2)

In [ ]:
def load_data(num_words, sequence_length, test_size=0.25, oov_token=None):
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
    num_classes=528
    y = to_categorical(np.asarray(label), num_classes)
   
   
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=1)
    data = {}
    data["X_train"] = X_train
    data["X_test"]= X_test
    data["y_train"] = y_train
    data["y_test"] = y_test
    data["tokenizer"] = tokenizer
    return data

In [ ]:
history = model.fit(df['X_train'], df['y_train'], validation_data=(df['X_test'], df['y_test']), epochs=10, verbose=2)

In [ ]:
def load_data(num_words, sequence_length, test_size=0.25, oov_token=None):
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
    num_classes=528
    y = to_categorical(np.asarray(label), num_classes)
   
   
   
    tokenizer.fit_on_texts(corpus)

    X = tokenizer.texts_to_sequences(corpus)
    X = np.array(X)
    X = pad_sequences(X, maxlen=sequence_length)
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=1)
    data = {}
    data["X_train"] = X_train
    data["X_test"]= X_test
    data["y_train"] = y_train
    data["y_test"] = y_test
    data["tokenizer"] = tokenizer
    return data